In [27]:
import pandas as pd
import json
from datetime import datetime
import holidays
import os
from datetime import timedelta

In [9]:
#df filtered by houston, needed to filter by state = TX (NOT IMPLEMENTED)

subway = pd.read_csv('/Users/mazcu/Downloads/Subway_Houston_days.csv', encoding="utf-8")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7,8,9,17,19,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
def is_weekend(df):
    df['date'] = pd.to_datetime(df['date']) 

    df['is_weekend'] = df['date'].dt.day_name()
    df.loc[df['is_weekend'] == "Saturday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Sunday", 'is_weekend'] = 1
    df.loc[df['is_weekend'] == "Monday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Tuesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Wednesday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Thursday", 'is_weekend'] = 0
    df.loc[df['is_weekend'] == "Friday", 'is_weekend'] = 0

    return df

def is_holiday(df):
    holi = holidays.CountryHoliday('US', prov="Houston", state='TX')
    df["is_holiday"] = [1 if d in holi else 0 for d in df["date"]]
    return df

def last_day(df):
    stores = set(df['placekey'])
    i = 0
    for store in stores:
        new = df[df['placekey'] == store]
        l = []
        
        #for row in new.iterrows():
        yesterday = new['date'] - timedelta(days = 1)
        yesterday = yesterday.dt.strftime('%Y-%m-%d')
        set_raro = set(new['date'].dt.strftime('%Y-%m-%d'))
        for day in yesterday:
            if day in set_raro:
                visits = list(new[new['date'] == day]['visits'])
                l.append(visits[0])
                
            else:
                l.append(0)
        
        new['visits_last_day'] = l

        if i == 0:
            old = new
            i += 1
        else:
            old = pd.concat([old, new])
    return old

def last_week(df):
    stores = set(df['placekey'])
    i = 0
    for store in stores:
        new = df[df['placekey'] == store]
        l = []
        
        #for row in new.iterrows():
        yesterday = new['date'] - timedelta(days = 7)
        yesterday = yesterday.dt.strftime('%Y-%m-%d')
        set_raro = set(new['date'].dt.strftime('%Y-%m-%d'))
        for day in yesterday:
            if day in set_raro:
                visits = list(new[new['date'] == day]['visits'])
                l.append(visits[0])
                
            else:
                l.append(0)
        
        new['visits_last_week'] = l

        if i == 0:
            old = new
            i += 1
        else:
            old = pd.concat([old, new])
    return old

subway = is_weekend(subway)
subway = is_holiday(subway)
subway = last_day(subway)
subway = last_week(subway)


<ipython-input-10-f7b8fb7eba20>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['visits_last_day'] = l
<ipython-input-10-f7b8fb7eba20>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['visits_last_week'] = l


In [11]:
subway['date'].count

<bound method Series.count of 3441    2020-01-01
3442    2020-01-02
3443    2020-01-03
3444    2020-01-04
3445    2020-01-05
           ...    
90590   2021-03-27
90591   2021-03-28
90592   2021-03-29
90593   2021-03-30
90594   2021-03-31
Name: date, Length: 92796, dtype: datetime64[ns]>

import os

for filename in os.listdir('/Users/mazcu/Downloads/safegraph_open_census_data/data/'): 
    print(filename)
    cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/'+filename, encoding="utf-8")
    try:
        print(filename, cosa['B19013e1'])
    except:
        pass

In [29]:
cosa = pd.read_csv('/Users/mazcu/Downloads/safegraph_open_census_data/data/cbg_b19.csv', encoding="utf-8", dtype = {"census_block_group": "category"})


In [13]:
df = subway

df['poi_cbg'] = df['poi_cbg'].map(int)
df['poi_cbg'] = df['poi_cbg'].map(str)


In [18]:
new = df[df['region'] == 'TX']
cbgs = set(new['poi_cbg'])

In [86]:
li = list(cosa['census_block_group'])
li = [str(i) for i in li]
for c in cbgs:

    if c in li:
        print('SÍ', c) #está    :D
    else:
        print('NO', c) #no está :,(

SÍ 482014323004
SÍ 482015517031
SÍ 482013501002
SÍ 482013330002
SÍ 482012220001
SÍ 482015401002
SÍ 482012329002
SÍ 482015340031
SÍ 482015410011
SÍ 482012323021
SÍ 482013402012
SÍ 482012214001
SÍ 482013119001
SÍ 482013112001
SÍ 482013332013
SÍ 482015217004
SÍ 482015510001
SÍ 482012227002
SÍ 482015530021
SÍ 482013128002
SÍ 482014107022
SÍ 482014301003
SÍ 482015508002
SÍ 482014401001
SÍ 482012520001
SÍ 482015417003
SÍ 482014543014
SÍ 482012313002
SÍ 482013109004
SÍ 482013340011
SÍ 482015114002
SÍ 482015336001
SÍ 482014532001
SÍ 482013338001
SÍ 482013319001
SÍ 482013126002
SÍ 482015328001
SÍ 482014211022
SÍ 482014319002
SÍ 482015515002
SÍ 482013321002
SÍ 482015522003
SÍ 482015432001
SÍ 482013125002
SÍ 482015409022
SÍ 482015408002
SÍ 482015527001
SÍ 482013102001
SÍ 482012115005
SÍ 482015204001
SÍ 482015115005
SÍ 482014229002
SÍ 482015321001
SÍ 482015341003
SÍ 482012117001
SÍ 482013339022
SÍ 482015514003
SÍ 482011000003
SÍ 482014521002
SÍ 482014217001
SÍ 482014505001
SÍ 482012226002
SÍ 48201

In [30]:
cosa = cosa[cosa['census_block_group'].isin(cbgs)]
cosa = cosa[]

,census_block_group,B19001e1,B19001m1,B19001e2,B19001m2,B19001e3,B19001m3,B19001e4,B19001m4,B19001e5,...,B19313Ee1,B19313Em1,B19313Fe1,B19313Fm1,B19313Ge1,B19313Gm1,B19313He1,B19313Hm1,B19313Ie1,B19313Im1
189207,481576701012,541,100,7,11,18,28,15,16,23,...,NaN,NaN,10563000.0,7103410.0,NaN,NaN,NaN,NaN,20749300.0,7541704.0
190093,482011000003,817,121,0,13,20,23,8,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,111937500.0,27931519.0,7882700.0,5634411.0
190101,482012105003,467,128,68,56,88,65,72,62,43,...,NaN,NaN,1559800.0,1788196.0,NaN,NaN,123500.0,182922.0,13121600.0,4874695.0
190121,482012112001,300,103,97,73,20,24,49,49,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190126,482012113003,428,112,65,55,76,61,8,11,27,...,NaN,NaN,2271800.0,3027855.0,NaN,NaN,1080200.0,1056435.0,6897400.0,5296956.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192114,482015527001,1123,232,0,13,0,13,0,13,0,...,NaN,NaN,NaN,NaN,NaN,NaN,79597100.0,25932065.0,11864100.0,7390032.0
192118,482015528002,455,114,27,45,0,13,32,52,16,...,NaN,NaN,NaN,NaN,NaN,NaN,57983900.0,23207223.0,NaN,NaN
192130,482015530021,819,137,47,35,34,39,44,44,6,...,NaN,NaN,3681700.0,3057008.0,NaN,NaN,39864200.0,13643389.0,17983300.0,7865013.0
192139,482015532004,176,96,42,67,0,13,51,81,46,...,NaN,NaN,2165400.0,3167126.0,NaN,NaN,1275000.0,1591848.0,2525700.0,3149112.0


,census_block_group,B19001e1,B19001m1,B19001e2,B19001m2,B19001e3,B19001m3,B19001e4,B19001m4,B19001e5,...,B19313Ee1,B19313Em1,B19313Fe1,B19313Fm1,B19313Ge1,B19313Gm1,B19313He1,B19313Hm1,B19313Ie1,B19313Im1


In [22]:
cosa['census_block_group']

Series([], Name: census_block_group, dtype: int64)